# Regional scale example

<div class="alert alert-block alert-info"> You will need to run the `meshGen` notebook to produce the topographic file required to run the model!</div>

All the input parameters are specified in the `input_bluemountains.yml` file.

## Input file

Input files for **eSCAPE** are based on [YAML](https://circleci.com/blog/what-is-yaml-a-beginner-s-guide/) syntax.

```YAML
name: Regional model

domain:
    filename: ['data/blueMountains.vtk','Z'] 
    flowdir: 5
    bc: slope

time:
    start: 0
    end: 20000
    tout: 1000.
    dt: 500.

sea:
    position: 100.

climate:
    - start: -10000000.
      uniform: 1.0

sp_br:
    Kbr: 5.e-6

sp_dep:
    Ff: 0.2

diffusion:
    hillslopeK: 1.e-5
    sedimentK:: 1.e4
    
output:
    dir: 'blueMountains'
    makedir: False

```

#### Parameters 

+ `domain`: definition of the unstructured grid containing the vtk grid `filename` and the associated field (here called `Z`) as well as the flow direction method to be used `flowdir` that takes an integer value between 1 (for SFD) and 12 (for Dinf) and the boundary conditions (`bc`: 'flat', 'fixed' or 'slope')

+ `time`: the simulation time parameters defined by `start`, `end`, `tout` (the output interval) and `dt` (the internal time-step).

Follows the optional forcing conditions:

+ `sea`: the sea-level declaration with the relative sea-level `position` (m) and the sea-level `curve` which is a file containing 2 columns (time and sea-level position).

+ `climatic` & `tectonic` have the same structure with a sequence of events defined by a starting time (`start`) and either a constant value (`uniform`) or a `map`.

Then the parameters for the surface processes to simulate:

+ `sp_br`: for the _stream power law_ with a unique parameter `Kbr` representing the The erodibility coefficient which is scale-dependent and its value depend on lithology and mean precipitation rate, channel width, flood frequency, channel hydraulics. It is worth noting that the coefficient _m_ and _n_ are fixed in this version and take the value 0.5 & 1 respectively. In this example we consider that all eroded sediments  are transported as fine suspension `Ff`=1 and as such will never be redeposited.

+ `diffusion`: hillslope, stream and marine diffusion coefficients. `hillslopeK` sets the _simple creep_ transport law which states that transport rate depends linearly on topographic gradient. The marine sediment are transported based on a diffusion coefficient `sedimentK`. 

Finally, you will need to specify the output folder:

+ `output`: with `dir` the directory name and the option `makedir` that gives the possible to delete any existing output folder with the same name (if set to False) or to create a new folder with the give `dir` name plus a number at the end (e.g. outputDir_1 if set to True)

### Using Jupyter notebook environment

For small models it is possible to use the notebook environment directly and run the following set of commands:

```python
import eSCAPE

# Reading input file
model = eSCAPE.LandscapeEvolutionModel(input_globe.yml)

# Running model
model.runProcesses()

# Running model
model.destroy()
```

### Using Python file

Here we will use a Python script called `run_escape.py` located in the same folder as your input file. 

```python
import argparse
import eSCAPE as sim

# Parsing command line arguments
parser = argparse.ArgumentParser(description='This is a simple entry to run eSCAPE model.',add_help=True)
parser.add_argument('-i','--input', help='Input file name (YAML file)',required=True)
parser.add_argument('-v','--verbose',help='True/false option for verbose', required=False,action="store_true",default=False)
parser.add_argument('-l','--log',help='True/false option for PETSC log', required=False,action="store_true",default=False)

args = parser.parse_args()
if args.verbose:
  print("Input file: {}".format(args.input))
  print(" Verbose is on? {}".format(args.verbose))
  print(" PETSC log is on? {}".format(args.log))

# Reading input file
model = sim.LandscapeEvolutionModel(args.input,args.verbose,args.log)

# Running model
model.runProcesses()

# Cleaning model
model.destroy()
```

This script is basically equivalent to what you will have done in the Jupyter environment but can also be ran on multiple processors using the `mpirun` command as shown below:

In [7]:
!cd /live/share/eSCAPE/; python setup.py install

running install
running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "eSCAPE._fortran" sources
f2py options: []
  adding 'build/src.linux-x86_64-2.7/build/src.linux-x86_64-2.7/fortran/fortranobject.c' to sources.
  adding 'build/src.linux-x86_64-2.7/build/src.linux-x86_64-2.7/fortran' to include_dirs.
Reading .f2py_f2cmap ...
	Mapping "integer(kind=HANDLE_KIND)" to "long"
	Mapping "integer(kind=handle_kind)" to "long"
Successfully applied user defined changes from .f2py_f2cmap
build_src: building npy-pkg config files
running build_py
copying eSCAPE/flow/surfprocplex.py -> build/lib.linux-x86_64-2.7/eSCAPE/flow
running build_ext
customize UnixCCompiler
customize UnixCCompiler using build_ext
get_default_fcompiler: matching types: '['gnu95', 'intel', 'lahey', 'pg', 'absoft', 'na

In [ ]:
!mpirun -np 4 python run_escape.py -i input_bluemountains.yml

89 100.0
89 100.0
8989 100.0
 100.0
--- Initialisation Phase (11.10 seconds)
+++ Output Simulation Time: 0.00 years
--- Computational Step (8.06 seconds)
--- Computational Step (7.58 seconds)
+++ Output Simulation Time: 1000.00 years
--- Computational Step (7.98 seconds)
--- Computational Step (7.66 seconds)
+++ Output Simulation Time: 2000.00 years
--- Computational Step (8.56 seconds)
--- Computational Step (7.77 seconds)
+++ Output Simulation Time: 3000.00 years
--- Computational Step (8.76 seconds)
--- Computational Step (8.17 seconds)
+++ Output Simulation Time: 4000.00 years
--- Computational Step (8.05 seconds)
--- Computational Step (8.31 seconds)
+++ Output Simulation Time: 5000.00 years
--- Computational Step (9.02 seconds)
--- Computational Step (9.03 seconds)
+++ Output Simulation Time: 6000.00 years
--- Computational Step (8.59 seconds)
--- Computational Step (8.81 seconds)
+++ Output Simulation Time: 7000.00 years
--- Computational Step (8.47 seconds)
--- Computational St

## Visualisation

Model outputs consist of a time series of hdf5 files that can be visualised with Paraview. A Paraview state is provided in the folder (`paravis.pvsm`) but might only work with the latest version of Paraview (5.6.0-RC1) and with a Mac...

In cases where you are using another version you will only have to load the top file in the output folder named `escape.xdmf`. 

For this particular example you will then use the `Wrap by Scalar` filter from Paraview to visualise the output in 3D.

The following parameters can be visualised from the output files:

+ `Z`: elevation
+ `ED`: erosion deposition
+ `SL`: sediment load
+ `FA`: flow accumulation


***